# Chicago to Cleveland
This notebook selects the route from the 47th Street Intermodal yard in Chicago IL to the Maple Heights Intermodal yard in Cleveland OH.

In [1]:
import geopandas as gpd
import networkx as nx
from shapely import geometry, ops
import numpy as np
import matplotlib.pyplot as plt
import requests
import pandas as pd
import folium
import time
from requests.auth import AuthBase
import os
from scipy.spatial import distance

In [2]:
URL ="http://score-web-1:8000"
URL1 = URL+"/api-token-auth/"
payload = {'username':'locomotives', 'password':'locomotives'}
URL2 = URL+"/api/line/add/"
URL3 = URL+'/api/railroad/'
URL4 = URL + "/api/line/"
URL5 = URL + "/api/route/add/"
URL6 = URL + "/api/yard/all/"
URL7 = URL + "/api/yard/"
URL8 = URL + "/api/route/detail/"
URL9 = URL + "/api/route/elevations/update/"
URL10 = URL + "/api/yard/add/"

In [3]:
t = requests.post(URL1, data=payload )
token = t.json().get('token')

In [4]:
class TokenAuth(AuthBase):
    """ Implements a custom authentication scheme. """

    def __init__(self, token):
        self.token = token

    def __call__(self, r):
        """ Attach an API token to a custom auth header. """
        r.headers['Authorization'] = "Token " + f'{self.token}'
        return r

Lets do a  quick test to make sure we can commmunicate with the server and grab the list of railroads.

In [5]:
r = requests.get(URL3, auth=TokenAuth(token))
railroads=r.json()['results']
railroads

[{'id': 1, 'code': 'BNSF', 'name': 'Burlington Northern and Santa Fe'},
 {'id': 2, 'code': 'CN', 'name': 'Canadian National Railway'},
 {'id': 3, 'code': 'CP', 'name': 'Canadian Pacific Railway'},
 {'id': 4, 'code': 'CSXT', 'name': 'CSX Transportation'},
 {'id': 5, 'code': 'NS', 'name': 'Norfolk Southern Railway'},
 {'id': 6, 'code': 'KCS', 'name': 'Kansas City Southern Railway'},
 {'id': 7, 'code': 'UP', 'name': 'Union Pacific'}]

When defining a route, we need a starting and and ending location. These typically occur in railraod yards. We can grab the list of available yards from the server.

In [6]:
response = requests.get(URL6, auth=TokenAuth(token))
yards = response.json()['results']
yards

[{'id': 2, 'name': 'Rickenbacker'},
 {'id': 4, 'name': 'Norfolk'},
 {'id': 3, 'name': 'Croxton'},
 {'id': 6, 'name': 'Savannah'},
 {'id': 7, 'name': 'Macon'},
 {'id': 8, 'name': 'Tyrone'},
 {'id': 9, 'name': 'State College'},
 {'id': 5, 'name': 'Suffolk'},
 {'id': 10, 'name': 'Croxton'},
 {'id': 1, 'name': 'Landers Yard'},
 {'id': 11, 'name': 'Bethlehem Intermodal'},
 {'id': 12, 'name': 'Rutherford Intermodal'},
 {'id': 13, 'name': 'Pitcairn Intermodal'},
 {'id': 14, 'name': 'Maple Heights Intermodal'},
 {'id': 15, 'name': '47th Street Intermodal'}]

We want to start this route at the 47th Street yard.

In [7]:
response = requests.get(URL7+'15', auth=TokenAuth(token))
start = response.json()['results']
start

{'id': 15,
 'code': 'C47',
 'name': '47th Street Intermodal',
 'city': 'Chicago',
 'state': 'IL',
 'location': 414720,
 'owner': 5}

and end at the Maple Heights Yard in Cleveland

In [8]:
response = requests.get(URL7+'14', auth=TokenAuth(token))
end = response.json()['results']
end

{'id': 14,
 'code': 'MPH',
 'name': 'Maple Heights Intermodal',
 'city': 'Cleveland',
 'state': 'OH',
 'location': 450869,
 'owner': 5}

In [9]:
data = gpd.read_file("North_American_Rail_Network_Lines.geojson")

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed


In [10]:
ns_data = data[data[['rrowner1', 'rrowner2', 'rrowner3', 'trkrghts1', 'trkrghts2', 'trkrghts3', 'trkrghts4', 'trkrghts5', 'trkrghts6', 'trkrghts7', 'trkrghts8', 'trkrghts9']].isin(['NS']).any(axis=1)]
#ns_data = data

## Determine the shortest route
We use the networkx library to determine the "optimal" route between two nodes on a graph. For this optimization, the graph consists of all nodes and "edges" or lines for which NOrfolk Southern has owns or has rights. This was determiend above.

In [11]:
from_nodes = list(ns_data.frfranode)

In [12]:
to_nodes = list(ns_data.tofranode)

The nodes in the graph is a union of all of the to and from nodes in the dataset.

In [13]:
all_nodes = list(set(from_nodes) | set(to_nodes))

The next step is to actually create a graph.

In [14]:
G=nx.Graph()

In [15]:
G.add_nodes_from(all_nodes)

We need to add the edges between the nodes and give them a weighting of the distance between the nodes - this will result in the shortest route.

In [16]:
edges = ns_data[['frfranode','tofranode', 'miles']].to_records(index=False).tolist()

In [17]:
G.add_weighted_edges_from(edges)

In [18]:
route = nx.astar_path(G, start['location'], end['location'])

In [19]:
route_data = ns_data[(ns_data['frfranode'].isin(route) & ns_data['tofranode'].isin(route))]

In [20]:
fig = folium.Figure(height=600)
map = folium.Map(location=[40.75, -85.0], zoom_start=10,tile=None)
folium.TileLayer(tiles='http://{s}.tiles.openrailwaymap.org/standard/{z}/{x}/{y}.png', attr='OpenStreetMap attribution').add_to(map)
folium.TileLayer(tiles='http://{s}.google.com/vt/lyrs=s,h&x={x}&y={y}&z={z}', attr='Google Maps').add_to(map)
folium.GeoJson(route_data).add_to(map)
map.add_to(fig)

This route takes us through Toledo. We want to route through Fort Wayne instead so remvoe a node from this route to help force it to go through Fort Wayne.

In [21]:
G.remove_node(437112)

In [22]:
route2 = nx.astar_path(G,start['location'], end['location'])

In [23]:
route2_data = ns_data[(ns_data['frfranode'].isin(route2) & ns_data['tofranode'].isin(route2))]

In [24]:
fig = folium.Figure(height=600)
map = folium.Map(location=[40.75, -85.0], zoom_start=10,tile=None)
folium.TileLayer(tiles='http://{s}.tiles.openrailwaymap.org/standard/{z}/{x}/{y}.png', attr='OpenStreetMap attribution').add_to(map)
folium.TileLayer(tiles='http://{s}.google.com/vt/lyrs=s,h&x={x}&y={y}&z={z}', attr='Google Maps').add_to(map)
folium.GeoJson(route2_data).add_to(map)
map.add_to(fig)

In [25]:
len(route2_data.geometry.get_coordinates())

3219

This route looks good, we can now move onto the next phase, simplification and segmentation. There are 3219 cooridnataes in this route. This can be reduced with simplify. Before we simplify, we need to convert the coordinate reference system to be in meters.

In [27]:
route2_data = route2_data.to_crs(3857)
simple_route = route2_data.simplify(1, preserve_topology=False)

To better support the estimation of gradient (and may also assist in curvature), we should segmentize the data to have a maximum distance between points.

In [28]:
simple2_route = simple_route.segmentize(max_segment_length=200)
simple2_route = simple2_route.to_crs(4326)
len(simple2_route.geometry.get_coordinates())

4607

In [29]:
len(simple2_route)

277

We get the necessary altitudes at this point. This is a relative long process of querrying. The data in the dataframe consists of a number of line segments. In this case there are 277.

In [30]:
def curve(p1, p2, p3):
    v1=p2-p1
    v2=p3-p2
    v3=p1-p3
    d1=np.linalg.norm(v1)
    d2=np.linalg.norm(v2)
    d3=np.linalg.norm(v3)
    num = 1746.375*2*np.linalg.norm(np.cross(v1,v2))
    d = num/(d1*d2*d3)
    if d<0.01:
        d=0.0
    return d

In [31]:
route2_xy = route2_data.copy()

In [32]:
route2_ll = route2_data.to_crs(4326)

In [47]:
def get_alt(lat, lng):
    # USGS Approach
    res = []
    for i in range(len(lng)):
        lt = lat[i]
        lg = lng[i]
        url = "https://epqs.nationalmap.gov/v1/json?x=" + str(lg) + "&y=" + str(lt) + "&units=Meters&wkid=4326&includeDate=False"
        response = requests.get(url)
        print(response.status_code)
        # print(response.text)
        # should put in a test for response status before returning an actual value
        if (response.status_code == 200):
            # print(response.json())
            if 'Invalid' in response.text:
                result = -1.0
            else:
                result = float(response.json().get('value',0.0))
        else:
            result = 0.0
        res.append(result)
        print(lt, lg, result)
    return res

In [34]:
codes = [d['code'] for d in railroads]
codes

['BNSF', 'CN', 'CP', 'CSXT', 'NS', 'KCS', 'UP']

In [35]:
def create_row(rowxy, rowll, rights, url, token=None):
    rights = []
    for code in codes:
        if rowxy[['rrowner1', 'rrowner2', 'rrowner3', 'trkrghts1', 'trkrghts2', 'trkrghts3', 'trkrghts4', 'trkrghts5', 'trkrghts6', 'trkrghts7', 'trkrghts8', 'trkrghts9']].isin([code]).any():
            rights.append(code)
    p = np.array(rowxy.geometry.coords)
    pxy = p[:,0:2]
    # we want the interpoint distance between the points - the offset of 1, diagonal of the cdist matrix
    dist = np.diagonal(distance.cdist(pxy, pxy), offset=1)
    lng,lat = rowll.geometry.xy
    ele=get_alt(lat, lng)
    lnglat = np.array(rowll.geometry.coords)
    dele = np.diff(ele)
    gradient = np.divide(dele, dist)
    curvature=[]
    if (len(pxy)>2):
        for i in range(len(pxy)-2):
            curvature.append(curve(pxy[i],pxy[i+1],pxy[i+2]))
        curvature.append(curvature[-1])
    else:
        curvature = [0.0]
    line = {
        "fra_id" : rowxy['fraarcid'],
        "from_node" : rowxy['frfranode'],
        "to_node" : rowxy['tofranode'],
        "length" :dist.sum(),
        "rights" : rights,
        "net": rowxy['net'],
        "xy": pxy.tolist(),
        "elevation": ele,
        "lnglat": lnglat.tolist(),
        "gradient": gradient.tolist(),
        "curvature": curvature,
        "distance":dist.tolist()
    }
    requests.post(url, data=line, auth=TokenAuth(token))
    print(line['fra_id'])
        

we need to explode the multiliestrings that are being used now to be linestrings

In [36]:
route2_xy.geometry=simple2_route.to_crs(3857)
route2_xy2 = route2_xy.explode(index_parts=True)

In [37]:
np.array(route2_xy2.iloc[0].geometry.coords)

array([[-9348464.41851551,  5027826.08330305],
       [-9348406.09522865,  5027827.6896273 ]])

Test to add one row

In [38]:
route2_ll.geometry=simple2_route
route2_ll2 = route2_ll.explode(index_parts=True)
np.array(route2_ll2.iloc[0].geometry.coords)

array([[-83.9786847 ,  41.10489566],
       [-83.97816077,  41.10490653]])

In [48]:
create_row(route2_xy2.iloc[0], route2_ll2.iloc[0], [5], URL2, token)

200
41.10489566100006 -83.97868470199995 230.07069397
200
41.104906534000065 -83.97816077499994 230.147125244
390312


In [58]:
for i in range(len(route2_xy2)):
    if i>1:
        # first test to see if it already exists
        fra_id = route2_xy2.iloc[i]['fraarcid']
        response = requests.get(URL4+str(fra_id), auth=TokenAuth(token))
        # if (response.status_code == 204):
        print(i, route2_xy2.iloc[i]['fraarcid'])
        create_row(route2_xy2.iloc[i], route2_ll2.iloc[i], [5], URL2, token)
        # print(route2_xy2.iloc[i][['fraarcid','frfranode', 'tofranode','net','miles']]) 

2 390346
200
41.37396287600007 -82.07908560099997 223.34312439
200
41.37361901147667 -82.08076703299997 223.290100098
200
41.37327514513553 -82.08244846499997 223.023727417
504
41.372931276976644 -82.08412989699995 0.0


KeyboardInterrupt: 